In [ ]:
# webscrape the reviews of an amazon product when given a url as input
# Take those reviews and covnert it into a text object
# Create a word cloud from that text object

In [8]:
# Take user input for the url and get the review list url
import re

#Getting user input
url = input("Enter the URL: ")

# Splitting the url and creating the link to the reviews list
url_string = re.split('/dp/', url)
product = url_string[0]
product_id = re.split(r'\?', url_string[1])[0] #Note that we can't do '?' since ? is an expression in re
review_url = product + '/product-reviews/' + product_id + '/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

Enter the URL:  https://www.amazon.com/LUPIT-POLE-Dance-Pole-Classic/dp/B082B98S17/ref=sr_1_8?crid=2ODBG4PH6QMFS&keywords=pole+pole+dancing&qid=1684697154&sprefix=pole+pole+dancing%2Caps%2C135&sr=8-8


In [9]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pandas as pd

session = AsyncHTMLSession()
review_data = []

while review_url:
    r = await session.get(review_url)
    await r.html.arender(sleep=5)
    soup = BeautifulSoup(r.html.html, 'lxml')
    reviews = soup.find_all("div", {"data-hook": "review"})
    for x in reviews:
            if x:
                title_element = x.find('a', {'data-hook': 'review-title'})
                body_element = x.find('span', {'data-hook': 'review-body'})

                title = title_element.text.strip() if title_element else ''
                body = body_element.text.strip() if body_element else ''

                review = {
                    'title': title,
                    'body': body
                }
                # print(review)
                review_data.append(review)
                print(review)
    next_page_element = soup.find(class_="a-last").find('a')

    if next_page_element is not None:
        next_page = 'https://www.amazon.com/' + next_page_element['href']
    else:
        next_page = None
    review_url = next_page

{'title': "THIS is IT!  Popcorn ceilings and X-Pole's atrocious wait times can't even stand in your way now", 'body': "Where to start?I've been wanting a home pole for ages, and have had my eye on an X-Pole for years.  Now that it was time to finally pull the trigger, and I jumped into my research.  While XP is still easily the most popular, I was picking up on a LOT of recent customer service issues online (customers having their card info stolen, double charges, endless processing times), all without being able to easily reach a customer support agent.  Lots of these people stated that they wished they had gone with Lupit pole.Lupit really checked out and everything I read about the quality and process seemed really great-- and then I noticed they even had an Amazon store.  I placed my order on a Monday evening, and the pole of my dreams arrived that Friday!  The shipping was a little expensive, but it's a big and HEAVY box, so I get it.  They also delivered it to my upstairs apartme

In [10]:
# Create a dataframe from the text data and clean it 
import pandas as pd

text_data = pd.DataFrame(review_data)
text_data.dropna(inplace=True)
text_data.head()

,title,body
0,THIS is IT! Popcorn ceilings and X-Pole's atr...,Where to start?I've been wanting a home pole f...
1,"Great pole, bought an extension","Pole is wonderful, easy to put together. I’m a..."
2,Easy to install,"I had an issue with the delivery, but the sell..."
3,Love mine,Arrived quickly and easy to install. I love it!
4,Very disappointing,This was a major disappointing purchase. My p...


In [62]:
# Need to get all the strings from each column, and then join them together into one large string
title_list = text_data['title'].to_list()
body_list = text_data['body'].to_list()

# Transforming the data into a text object

# lines = title_list + body_list
text = " ".join(x.strip() for x in body_list)
text = re.sub('The media could not be loaded', '', text)

# Remove all whitespace using re.sub()
pattern = r'[\t\n]+'  # '\t' matches tabs and '\n' matches newlines, + means more than one
text = re.sub(pattern, '', text)

emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
text = re.sub(emoji, '', text) # removing all emojis

Where to start?I've been wanting a home pole for ages, and have had my eye on an X-Pole for years.  Now that it was time to finally pull the trigger, and I jumped into my research.  While XP is still easily the most popular, I was picking up on a LOT of recent customer service issues online (customers having their card info stolen, double charges, endless processing times), all without being able to easily reach a customer support agent.  Lots of these people stated that they wished they had gone with Lupit pole.Lupit really checked out and everything I read about the quality and process seemed really great-- and then I noticed they even had an Amazon store.  I placed my order on a Monday evening, and the pole of my dreams arrived that Friday!  The shipping was a little expensive, but it's a big and HEAVY box, so I get it.  They also delivered it to my upstairs apartment door; what a pleasant surprise!  Assembly was a breeze using one of their YouTube videos-- and this is with popcorn 

In [69]:
# Pre-processing the data
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Lowercasing and removing punctuation
text = "".join(i for i in list(text) if i not in string.punctuation).lower()

# Tokenizing the data
text_tokens = word_tokenize(text)

# Removing the stopwords
stopwords = set(stopwords.words('english'))
text_words = [word for word in text_tokens if word not in stopwords]

In [ ]:
# Compute the frequency of each word and combine common adjacent phrases 